<a href="https://colab.research.google.com/github/flokoafon/WQU_project/blob/main/WQU_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sn
import io
from google.colab import files

In [27]:
url = "https://raw.githubusercontent.com/flokoafon/WQU_project/main/ProjectData.xlsx"
series = pd.read_excel(url)
series.head()

,Date,DPI,IPI,COP,SIR,LTIR,NGP
0,2000-01-01,6835.600098,76.49362,27.01,3.09,5.539524,NaN
1,2000-02-01,7644.549805,78.39347,29.30,3.14,5.508571,NaN
2,2000-03-01,7599.390137,78.49346,29.89,3.36,5.333043,NaN
3,2000-04-01,7414.680176,79.49338,25.54,3.53,5.225556,NaN
4,2000-05-01,7109.669922,81.39322,28.81,3.87,5.378182,NaN


In [ ]:
#Data importation
uploades = files.upload()


Saving ProjectData.csv to ProjectData.csv


In [ ]:
series = pd.read_csv(io.BytesIO(uploades['ProjectData.csv']), sep=";", index_col="Date", decimal=",")
series

,DPI,IPI,COP,SIR,LTIR,NGP
Date,,,,,,
01/01/2000,6835.600098,76.5,27.01,3.09,5.54,NaN
01/02/2000,7644.549805,78.4,29.30,3.14,5.51,NaN
01/03/2000,7599.390137,78.5,29.89,3.36,5.33,NaN
01/04/2000,7414.680176,79.5,25.54,3.53,5.23,NaN
01/05/2000,7109.669922,81.4,28.81,3.87,5.38,NaN
...,...,...,...,...,...,...
01/11/2021,15100.129880,94.3,79.18,-0.52,-0.31,NaN
01/12/2021,15884.860350,95.8,71.69,-0.58,-0.38,NaN
01/01/2022,15471.200200,96.4,82.98,-0.50,-0.11,NaN
